In [113]:
from grab import Grab
import json

g = Grab()

def get_stock_links():
    url = 'http://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/constituents-indices.html?&page={}'
    pages = [url.format(page) for page in range(1, 80)]
    stocks = []
    count_urls = 0
    
    for page in pages:
        g.go(page)
        for elem in g.doc.select('//*[@id="fullcontainer"]/div/table/tbody/tr/td[2]/a'):
            count_urls+=1
            stocks.append('http://www.londonstockexchange.com' + elem.attr('href'))
    print("found: " + str(count_urls) + " stocks")
    return stocks

def get_contract_specs(stock_lst):
    stocks_result = {}
    
    for link in stock_lst:
        g.go(link)
        # print(link)
        try:
            for w in g.doc.select('//*[@id="box-testata"]/div/h1'):
                # print(w.text())
                # todo IHE regex, 8-9 digits strip
                _data = {'description': (w.text().split(' ', maxsplit=1))[1],
                        'data':'empty',
                        'IHE': (str(link.split(".")[-2].split('/')[-1]))[:-8],
                        'symbol':  (w.text().split(' ', maxsplit=1))[0].split('.')[0] + '.LD',
                        'link': link}
                stocks_result[(str(link.split(".")[-2].split('/')[-1]))] = _data
                
        except BaseException as e:
            print(e)
    return stocks_result

def parse_data(stock_dict):
    
    _headers = {'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4',
           'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
           'Content-Type': 'application/json; charset=UTF-8',
           'Accept': 'application/json, text/javascript, */*; q=0.01',
           'Referer': 'http://charts.londonstockexchange.com/chart/CompanySummaryChart2_image.aspx?mcode=TIT.3IN.LD&key=JE00BYR8GK67JEGBXSTMM&imageFormat=png',
           'X-Requested-With': 'XMLHttpRequest',
           'Connection': 'keep-alive'}

    _post = """{"request":{"SampleTime":"1mm",
    "TimeFrame":"1d",
    "RequestedDataSetType":"ohlc",
    "ChartPriceType":"price",
    "Key":"%s",
    "OffSet":-60,
    "FromDate":null,
    "ToDate":null,
    "UseDelay":true,
    "KeyType":"Topic",
    "KeyType2":"Topic",
    "Language":"en"}}"""

    _url = 'http://charts.londonstockexchange.com/WebCharts/services/ChartWService.asmx/GetPrices'

    for stock in stock_dict.keys():
        p = _post % (stock_dict[stock]['symbol'])
        try:
            g.go(url=_url, headers=_headers, post=p)
            # todo if today no data then we have string body
            stock_dict[stock]['data'] = str(g.doc.body)
        except BaseException as e:
            print(e)
            
    print("Parsed: " + str(len(stock_dict)) + " stocks")  
    return stock_dict


def save_to_file(data_save, filename):
    """save data to filename"""
    data_json = json.dumps(data_save, indent=4, sort_keys=True)
    
    with open("{}.txt".format(filename), "a") as f:
        f.write(data_json)
        f.write('\n')
    print("Saved to file {}".format(f.name))
    
if __name__ == '__main__':
    stock_links = get_stock_links()
    stocks = get_contract_specs(stock_links)
    data = parse_data(stocks)
    save_to_file(data, 'my_json_data_from_LSE')

found: 20 stocks
Parsed: 3 stocks
Saved to file my_json_data_from_LSE.txt
